In [48]:
# This is only for display purposes to monitor processing easier
import time, sys
from IPython.display import clear_output

def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

### This begins the video pre-processing

In [49]:
import os

# change this
image_instead_of_video = False

vid_file = 'starwars.mp4'
img_file = 'lego.png'

unique_colors = 32


#dont need to change these
repo_path = os.path.dirname(os.getcwd())
vid_path = '/vid/'
img_path = '/img/'

if image_instead_of_video == True:
    testfile = repo_path + img_path + img_file
else:
    testfile = repo_path + vid_path + vid_file

print("TESTFILE: " + testfile)


TESTFILE: /Users/jberry003/Documents/Dev/Projects/shine/vid/starwars.mp4


In [50]:
def round_colors(m, diff_colors):
    m = np.float32(m)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 500, 1)
    ret, label, center = cv2.kmeans(m, diff_colors, None, criteria, 30, cv2.KMEANS_PP_CENTERS)
    center = np.uint8(center)
    return center[label.flatten()]

In [51]:
import cv2
import numpy as np

def extract_dominant(pic_in, clusters):
    
    # normalize array to 0-1 instead of 0-255
    pic_norm = np.array(pic_in).astype('float32')/255
    
    # convert to LAB
    pic_lab = cv2.cvtColor(pic_norm, cv2.COLOR_BGR2LAB)
    
    # reshape - make it long and 2D
    pic_reshape = np.float32(pic_lab.reshape(-1,3))
    
    # cluster
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 5000, 5)
    flags = cv2.KMEANS_PP_CENTERS

    _, labels, palette = cv2.kmeans(pic_reshape, clusters, None, criteria, 30, flags)
    _, counts = np.unique(labels, return_counts=True)

    # most dominant cluster
    biggest_cluster = palette[np.argmax(counts)]
    
    # reshape to 3 channels
    biggest_cluster = np.float32(biggest_cluster.reshape(1,1,3))

    # convert back to BGR
    biggest_cluster = cv2.cvtColor(biggest_cluster, cv2.COLOR_LAB2BGR)
    
    # de-normalize back to bits
    biggest_cluster *= [255,255,255]
    #print('BIGGEST: ', biggest_cluster[0][0])
    return np.rint(biggest_cluster[0][0])

In [53]:
import datetime

def parse_to_array(vid, n):
    
    k_per_frame = 3
    
    update_progress(0)
    
    cap = cv2.VideoCapture(vid)
    fps = cap.get(cv2.CAP_PROP_FPS)
    tot = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    f = 0
    result = []
    times = []
    while(True):
        grabbed, frame = cap.read()
        if not grabbed:
            break
        f+=1
        sec = float(f)/fps
        if (f % n == 0 and f > -1):
            s = datetime.datetime.now()
            result.append(extract_dominant(frame, k_per_frame)[:3])
            times.append(sec)
            e = datetime.datetime.now()
            t = e - s
            #print(str(t.total_seconds() * 1000))
            update_progress(f / tot)
    cap.release()
    return result, times

In [ ]:
# take video file and take every nth frame, grabbing dominant color from each
training = parse_to_array(testfile, 5)

Progress: [###-----------------] 13.0%


In [27]:
result = round_colors(training[0], unique_colors)

In [35]:
import pandas as pd
df = pd.DataFrame({'t':np.asarray(training[1]),'b':result[:,0],'g':result[:,1],'r':result[:,2]})

In [36]:
df['helper'] = df['r'].astype(str) + "," + df['g'].astype(str) + "," + df['b'].astype(str)
df['counter'] = df.groupby((df['helper'] != df['helper'].shift(1)).cumsum()).cumcount()+1
df.head(5)

,t,b,g,r,helper,counter
0,1.84,67,60,52,"52,60,67",1
1,3.68,67,60,52,"52,60,67",2
2,5.52,67,60,52,"52,60,67",3
3,7.36,97,86,81,"81,86,97",1
4,9.20,215,174,150,"150,174,215",1


In [41]:
df['t'].max()

169.28

### This begins the Hue interaction - play video separately

In [37]:
# fyi the debug link is here: https://192.168.1.9/debug/clip.html
hue_ip = '192.168.1.9'
hue_user = '082Gdixhk1IWMX78CpodeFxBWSLbeEmO7PjpFFKy'

In [38]:
# Connect to the bridge with a particular username
from qhue import Bridge
bri = Bridge(hue_ip, hue_user)
lights = bri.lights 

In [39]:
from rgbxy import Converter
from rgbxy import GamutC # or GamutA, GamutB, GamutC

converter = Converter(GamutC)

def change_color(r,g,b):
    bri.lights[6].state(on=True, xy=converter.rgb_to_xy(r,g,b))
    
def light_off():
    bri.lights[6].state(on=False)

In [46]:
# loop through dataframe and show colors
import time
import datetime

light_off()

print("     in 5,...")
time.sleep(5)
print("GO!")
time.sleep(1)

i = 1
diff = 0

start = datetime.datetime.now()

while diff < df['t'].max():

    diff = (datetime.datetime.now() - start).total_seconds()
    row = df.iloc[(df['t'] - diff).abs().argsort()[:1]]
    r = row[['r']].values[0][0]
    g = row[['g']].values[0][0]
    b = row[['b']].values[0][0]
    
    if i == 1:
        newxy = converter.rgb_to_xy(r,g,b)
        oldxy = newxy
        change_color(r,g,b)
    else:
        oldxy = newxy
        newxy = converter.rgb_to_xy(r,g,b)
    
    if oldxy != newxy:
        print("CHANGED!  (" + str(r) + "," + str(g) + "," + str(b) + ")")
        change_color(r,g,b)
        time.sleep(1)
        
    i+=1
    

light_off()

     in 5,...
GO!
CHANGED!  (81,86,97)
CHANGED!  (150,174,215)
CHANGED!  (154,204,243)
CHANGED!  (33,55,51)
CHANGED!  (52,60,67)
CHANGED!  (160,188,219)
CHANGED!  (36,34,33)
CHANGED!  (52,60,67)
CHANGED!  (83,102,71)
CHANGED!  (97,112,75)
CHANGED!  (154,204,243)
CHANGED!  (160,188,219)
CHANGED!  (154,204,243)
CHANGED!  (36,34,33)
CHANGED!  (97,140,69)
CHANGED!  (231,212,204)
CHANGED!  (178,193,74)
CHANGED!  (210,207,210)
CHANGED!  (80,76,61)
CHANGED!  (99,88,75)
CHANGED!  (112,108,107)
CHANGED!  (107,95,85)
CHANGED!  (99,88,75)
CHANGED!  (107,95,85)
CHANGED!  (99,88,75)
CHANGED!  (140,128,117)
CHANGED!  (119,103,87)
CHANGED!  (89,103,31)
CHANGED!  (118,140,56)
CHANGED!  (99,88,75)
CHANGED!  (235,210,195)
CHANGED!  (118,140,56)
CHANGED!  (97,79,69)
CHANGED!  (135,110,92)
CHANGED!  (246,228,207)
CHANGED!  (97,79,69)
CHANGED!  (246,228,207)
CHANGED!  (118,140,56)
CHANGED!  (238,209,184)
CHANGED!  (118,140,56)
CHANGED!  (143,119,100)
CHANGED!  (154,187,233)
CHANGED!  (246,228,207)
CHANGED!